Needed Libraries

- Note about installing necessary packages
- Look up proper way to note libraries needed

In [ ]:
%load_ext dotenv

In [2]:
# Data Analysis Packages
import numpy as np
import pandas as pd
import matplotlib as mpl
import sklearn

# -- Utility Packages
import bs4

# ---- Folium for Map Generation
import folium

# ---- Packages used for API calls
import requests
import json

# ---- os for drawing out API keys from enviroment variables
import os

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
# Retrieve Secret and ID from Enviroment Variables
%dotenv

YELP_CLIENT_ID = os.getenv('YELP_CLIENT_ID')
YELP_API_KEY = os.getenv('YELP_API_KEY')

# Code for Foursquare API
# CLIENT_ID = os.getenv('CLIENT_ID')
# CLIENT_SECRET = os.getenv('CLIENT_SECRET')
# VERSION = '20180323'

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


Pull list of Cities from Orange County Website

In [3]:
r = requests.get('https://media.ocgov.com/about/infooc/links/oc/occities.asp')
soup = bs4.BeautifulSoup(r.text)

oc_city_list = []
for cityTag in soup.select('div.edtdiv ul > li div a'):
    city_string = str(cityTag.string)
    # Strip out 'City of'
    city_string = city_string[8:]
    oc_city_list.append(city_string)
oc_city_list

['Aliso Viejo',
 'Anaheim',
 'Brea',
 'Buena Park',
 'Costa Mesa',
 'Cypress',
 'Dana Point',
 'Fountain Valley',
 'Fullerton',
 'Garden Grove',
 'Huntington Beach',
 'Irvine',
 'La Habra',
 'La Palma',
 'Laguna Beach',
 'Laguna Hills',
 'Laguna Niguel',
 'Laguna Woods',
 'Lake Forest',
 'Los Alamitos',
 'Mission Viejo',
 'Newport Beach',
 'Orange',
 'Placentia',
 'Rancho Santa Margarita',
 'San Clemente',
 'San Juan Capistrano',
 'Santa Ana',
 'Seal Beach',
 'Stanton',
 'Tustin',
 'Villa Park',
 'Westminster',
 'Yorba Linda']

Restaurant list will be built from calls to Yelp's Fusion API

The Endpoint will be businesses search (https://www.yelp.com/developers/documentation/v3/business_search)

Test API Endpoint with Aliso Viejo

In [30]:
url = "https://api.yelp.com/v3/businesses/search"
headers = dict(
    Authorization = 'Bearer ' + YELP_API_KEY
)
params = dict(
    location = 'Aliso Viejo, CA',
    limit = 50,
    categories = 'Food',
    radius = 4000,
    sort_by = 'distance'
)

resp = requests.get(headers=headers, url=url, params=params)
data = json.loads(resp.text)

In [31]:
businesses = data['businesses']

aliso_viejo_spots = []

for business in businesses:
    if(business['location']['city'] == 'Aliso Viejo'):
        aliso_viejo_spots.append(business)

In [32]:
len(aliso_viejo_spots)

50

['hi']